# Graficar predicciones

## Usar env "pytorch_GPU_NPL_Ricard

In [3]:
import os
import joblib
import matplotlib.pyplot as plt
import numpy as np
import math
from sklearn.preprocessing import StandardScaler
import pandas as pd
import pickle
import plotly.graph_objects as go
import pandas as pd
import plotly
import plotly.io as pio
import dash
from dash import dcc, html


In [4]:
FIC_ENTRADA='prediccion_transformers.joblib'
RUTA_FICHEROS = r"C:\Users\jaume\Documents\Proyecto\datos"

# Parametros
WINDOW_SIZE=30
OFFSET=4

In [5]:
# Carga de datos
os.chdir(RUTA_FICHEROS)

with open(FIC_ENTRADA, 'rb') as file:
    data_dict = joblib.load(file)

# Recuperar DataFrame y scaler
#df_results = data_dict['df_results'].copy()
predictions_test = data_dict['predictions_test'].copy()


# Cargar desde el archivo
with open('train_test_val_scaler.joblib', 'rb') as file:
    data_dict = joblib.load(file)

# Recuperar DataFrame y scaler
df_train = data_dict['train'].copy() 
df_test = data_dict['test'].copy() 
df_val = data_dict['val'].copy() 
scaler = data_dict['scaler']              # Es el StandardScaler que entrenaste en [L,A,B,durac_log,tiempo_log,vel]

C:\Users\jaume\anaconda3\envs\pytorch_GPU_NLP_Ricard\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.5.2 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [6]:
df_test.head()

,L_scaled,A_scaled,B_scaled,tiempo_en_estado_scaled,velocidad_scaled,duracion_parada_robust_scaled,patron_0,patron_2,patron_3,patron_4,...,patron_88,patron_89,patron_90,patron_91,patron_92,patron_93,patron_94,estado_alta,estado_normal,estado_puesta
seq_time,,,,,,,,,,,,,,,,,,,,,
243863,-0.647141,0.290458,0.110672,-0.741699,1.374517,0.0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
243864,-0.645620,0.290659,0.110867,-0.735443,1.323209,0.0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
243865,-0.644100,0.290861,0.111061,-0.729233,1.513400,0.0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
243866,-0.642579,0.291062,0.111256,-0.723069,1.493978,0.0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
243867,-0.641058,0.291264,0.111451,-0.716951,1.529770,0.0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


# Codigo para ensamblar

In [7]:
# Crear un nuevo scaler solo con los primeros 3 componentes (L, A, B)
scaler_3d = StandardScaler()
scaler_3d.mean_ = scaler.mean_[:3]  # Primeras 3 medias (L, A, B)
scaler_3d.scale_ = scaler.scale_[:3]  # Primeras 3 desviaciones (L, A, B)

# Aplicar inverse_transform al DataFrame Y_pred_rnn
Y_pred_desescalado = scaler_3d.inverse_transform(predictions_test)

# Convertir el resultado a DataFrame (opcional, si prefieres mantenerlo en formato de tabla)
#Y_pred_desescalado = pd.DataFrame(Y_pred_desescalado, columns=["L", "A", "B"])

# Mostrar resultados
print(Y_pred_desescalado[:10])

[[ 59.02542  -26.903101  22.490723]
 [ 59.05364  -27.849857  22.61532 ]
 [ 59.106297 -27.205982  22.603184]
 [ 59.090347 -28.299269  22.73201 ]
 [ 59.10817  -26.910513  22.640148]
 [ 59.084164 -28.164202  22.789433]
 [ 59.124344 -26.063377  22.572205]
 [ 59.22045  -24.723936  22.372715]
 [ 59.229164 -26.94564   22.707243]
 [ 59.299496 -25.823292  22.551779]]


In [8]:
offset_inicial = WINDOW_SIZE + OFFSET - 1
offset_final = offset_inicial + len(Y_pred_desescalado)

df_test_pred = df_test.iloc[offset_inicial:offset_final].copy()
df_test_pred['L_pred'] = Y_pred_desescalado[:,0]
df_test_pred['A_pred'] = Y_pred_desescalado[:,1]
df_test_pred['B_pred'] = Y_pred_desescalado[:,2]

In [9]:
print(offset_inicial)
print(df_test.columns)
#print(df_scaled.columns)
print(df_test_pred.columns)
print(df_test[['L_scaled']].head(30))
print(df_test_pred[['L_scaled', 'L_pred']].head(30))
print(df_test_pred[['A_scaled', 'A_pred']].head(30))


33
Index(['L_scaled', 'A_scaled', 'B_scaled', 'tiempo_en_estado_scaled',
       'velocidad_scaled', 'duracion_parada_robust_scaled', 'patron_0',
       'patron_2', 'patron_3', 'patron_4',
       ...
       'patron_88', 'patron_89', 'patron_90', 'patron_91', 'patron_92',
       'patron_93', 'patron_94', 'estado_alta', 'estado_normal',
       'estado_puesta'],
      dtype='object', length=103)
Index(['L_scaled', 'A_scaled', 'B_scaled', 'tiempo_en_estado_scaled',
       'velocidad_scaled', 'duracion_parada_robust_scaled', 'patron_0',
       'patron_2', 'patron_3', 'patron_4',
       ...
       'patron_91', 'patron_92', 'patron_93', 'patron_94', 'estado_alta',
       'estado_normal', 'estado_puesta', 'L_pred', 'A_pred', 'B_pred'],
      dtype='object', length=106)
          L_scaled
seq_time          
243863   -0.647141
243864   -0.645620
243865   -0.644100
243866   -0.642579
243867   -0.641058
243868   -0.639537
243869   -0.638016
243870   -0.636496
243871   -0.634975
243872   -0.633454
2

In [10]:
initial_file="02_datos_limpios.pkl"
# Este lo usaré al final para ensamblar
with open(initial_file, 'rb') as file:
    df_inicial = pickle.load(file)

inicial_sin_parada="02_datos_limpios_sin_parada.pkl"
# Este lo usaré al final para ensamblar
with open(inicial_sin_parada, 'rb') as file:
    df_inicial_sin_parada = pickle.load(file)

In [11]:
df_inicial_sin_parada.head()

,tmin,tmed,tmax,horatmin,horatmax,hrMin,hrMedia,hrMax,horaHrMin,horaHrMax,...,A_base,B,B_base,Delta_E,patron_id,hour_sin,hour_cos,duracion_parada_anterior,tiempo_en_estado,seq_time
datetime,,,,,,,,,,,,,,,,,,,,,
2023-01-02 00:00:00.000,3.6,7.7,11.8,23:59:00,12:00:00,44.0,76.0,93.0,00:00:00,23:59:00,...,-63.254562,44.550268,44.550268,0.000000,0,0.0,1.0,0.0,0,0
2023-01-02 00:02:52.800,3.6,7.7,11.8,23:59:00,12:00:00,44.0,76.0,93.0,00:00:00,23:59:00,...,-63.254562,44.560268,44.550268,0.024495,0,0.0,1.0,0.0,1,1
2023-01-02 00:05:45.600,3.6,7.7,11.8,23:59:00,12:00:00,44.0,76.0,93.0,00:00:00,23:59:00,...,-63.254562,44.570268,44.550268,0.048990,0,0.0,1.0,0.0,2,2
2023-01-02 00:08:38.400,3.6,7.7,11.8,23:59:00,12:00:00,44.0,76.0,93.0,00:00:00,23:59:00,...,-63.254562,44.580268,44.550268,0.073485,0,0.0,1.0,0.0,3,3
2023-01-02 00:11:31.200,3.6,7.7,11.8,23:59:00,12:00:00,44.0,76.0,93.0,00:00:00,23:59:00,...,-63.254562,44.590268,44.550268,0.097980,0,0.0,1.0,0.0,4,4


In [12]:
df_merged = df_inicial_sin_parada.merge(
    df_test_pred[['L_pred', 'A_pred', 'B_pred']], 
    left_on='seq_time',  # Usa la columna en df_test
    right_index=True,    # Usa el índice en df_test_pred
    how='left'           # Mantiene todas las filas de df_test, rellenando NaN si no hay coincidencias
)


In [13]:
# Seleccionar las columnas de interés de df_filtered
columns_to_add = ["L_pred", "A_pred", "B_pred"]

# Combinar los DataFrames basados en el índice
df_final = df_inicial.join(df_merged[columns_to_add], how="left")

In [14]:


print(df_final.dropna(subset='L_pred'))

df_final.head()


                         tmin  tmed  tmax  horatmin  horatmax  hrMin  hrMedia  \
datetime                                                                        
2024-05-08 07:55:12.000   3.9  11.8  19.8  04:40:00  16:00:00   34.0     54.0   
2024-05-08 07:58:04.800   3.9  11.8  19.8  04:40:00  16:00:00   34.0     54.0   
2024-05-08 08:00:57.600   3.9  11.8  19.8  04:40:00  16:00:00   34.0     54.0   
2024-05-08 08:03:50.400   3.9  11.8  19.8  04:40:00  16:00:00   34.0     54.0   
2024-05-08 08:06:43.200   3.9  11.8  19.8  04:40:00  16:00:00   34.0     54.0   
...                       ...   ...   ...       ...       ...    ...      ...   
2024-07-01 21:10:04.800   9.3  14.6  19.9  05:00:00  17:40:00   49.0     55.0   
2024-07-01 21:12:57.600   9.3  14.6  19.9  05:00:00  17:40:00   49.0     55.0   
2024-07-01 21:15:50.400   9.3  14.6  19.9  05:00:00  17:40:00   49.0     55.0   
2024-07-01 21:18:43.200   9.3  14.6  19.9  05:00:00  17:40:00   49.0     55.0   
2024-07-01 21:21:36.000   9.

,tmin,tmed,tmax,horatmin,horatmax,hrMin,hrMedia,hrMax,horaHrMin,horaHrMax,...,B_base,Delta_E,patron_id,hour_sin,hour_cos,duracion_parada_anterior,tiempo_en_estado,L_pred,A_pred,B_pred
datetime,,,,,,,,,,,,,,,,,,,,,
2023-01-02 00:00:00.000,3.6,7.7,11.8,23:59:00,12:00:00,44.0,76.0,93.0,00:00:00,23:59:00,...,44.550268,0.000000,0,0.0,1.0,0.0,0,NaN,NaN,NaN
2023-01-02 00:02:52.800,3.6,7.7,11.8,23:59:00,12:00:00,44.0,76.0,93.0,00:00:00,23:59:00,...,44.550268,0.024495,0,0.0,1.0,0.0,1,NaN,NaN,NaN
2023-01-02 00:05:45.600,3.6,7.7,11.8,23:59:00,12:00:00,44.0,76.0,93.0,00:00:00,23:59:00,...,44.550268,0.048990,0,0.0,1.0,0.0,2,NaN,NaN,NaN
2023-01-02 00:08:38.400,3.6,7.7,11.8,23:59:00,12:00:00,44.0,76.0,93.0,00:00:00,23:59:00,...,44.550268,0.073485,0,0.0,1.0,0.0,3,NaN,NaN,NaN
2023-01-02 00:11:31.200,3.6,7.7,11.8,23:59:00,12:00:00,44.0,76.0,93.0,00:00:00,23:59:00,...,44.550268,0.097980,0,0.0,1.0,0.0,4,NaN,NaN,NaN


# Gráfico final predicciones

In [16]:
import plotly.graph_objects as go
import plotly.io as pio
import pandas as pd
import webbrowser
import os

# Filtrar solo las columnas necesarias para las gráficas
columns_to_plot = ['velocidad', 'L', 'L_pred', 'A', 'A_pred', 'B', 'B_pred']
df_filtered = df_final[columns_to_plot]

# Asegurar que el índice es datetime y ordenado
df_filtered = df_filtered.sort_index()

# Crear la figura para velocidad
fig_velocity = go.Figure()
fig_velocity.add_trace(go.Scatter(
    x=df_filtered.index,
    y=df_filtered['velocidad'],
    mode='lines',
    name='Velocidad',
    line=dict(color='blue', width=2)
))
fig_velocity.update_layout(
    title="Velocidad a lo largo del tiempo",
    xaxis_title="Tiempo",
    yaxis_title="Velocidad",
    template="plotly_white"
)

# Crear la figura para L y L_pred
fig_L = go.Figure()
fig_L.add_trace(go.Scatter(
    x=df_filtered.index,
    y=df_filtered['L'],
    mode='lines',
    name='L (Real)',
    line=dict(color='green', width=2)
))
fig_L.add_trace(go.Scatter(
    x=df_filtered.index,
    y=df_filtered['L_pred'],
    mode='lines',
    name='L (Predicción)',
    line=dict(color='red', dash='dash', width=2)
))
fig_L.update_layout(
    title="L y L_pred a lo largo del tiempo",
    xaxis_title="Tiempo",
    yaxis_title="L",
    template="plotly_white"
)

# Crear la figura para A y A_pred
fig_A = go.Figure()
fig_A.add_trace(go.Scatter(
    x=df_filtered.index,
    y=df_filtered['A'],
    mode='lines',
    name='A (Real)',
    line=dict(color='purple', width=2)
))
fig_A.add_trace(go.Scatter(
    x=df_filtered.index,
    y=df_filtered['A_pred'],
    mode='lines',
    name='A (Predicción)',
    line=dict(color='orange', dash='dash', width=2)
))
fig_A.update_layout(
    title="A y A_pred a lo largo del tiempo",
    xaxis_title="Tiempo",
    yaxis_title="A",
    template="plotly_white"
)

# Crear la figura para B y B_pred
fig_B = go.Figure()
fig_B.add_trace(go.Scatter(
    x=df_filtered.index,
    y=df_filtered['B'],
    mode='lines',
    name='B (Real)',
    line=dict(color='blue', width=2)
))
fig_B.add_trace(go.Scatter(
    x=df_filtered.index,
    y=df_filtered['B_pred'],
    mode='lines',
    name='B (Predicción)',
    line=dict(color='red', dash='dash', width=2)
))
fig_B.update_layout(
    title="B y B_pred a lo largo del tiempo",
    xaxis_title="Tiempo",
    yaxis_title="B",
    template="plotly_white"
)

# Crear una página HTML que contenga todas las gráficas
html_content = f"""
<html>
<head>
    <title>Visualización de Datos</title>
    <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
</head>
<body>
    <h1 style="text-align:center;">Visualización de Datos de Predicción</h1>

    <h3>Velocidad</h3>
    <div id="fig_velocity"></div>
    <script>
        var fig_velocity = {fig_velocity.to_json()};
        Plotly.newPlot('fig_velocity', fig_velocity.data, fig_velocity.layout);
    </script>

    <h3>L y L_pred</h3>
    <div id="fig_L"></div>
    <script>
        var fig_L = {fig_L.to_json()};
        Plotly.newPlot('fig_L', fig_L.data, fig_L.layout);
    </script>

    <h3>A y A_pred</h3>
    <div id="fig_A"></div>
    <script>
        var fig_A = {fig_A.to_json()};
        Plotly.newPlot('fig_A', fig_A.data, fig_A.layout);
    </script>

    <h3>B y B_pred</h3>
    <div id="fig_B"></div>
    <script>
        var fig_B = {fig_B.to_json()};
        Plotly.newPlot('fig_B', fig_B.data, fig_B.layout);
    </script>

</body>
</html>
"""

# Guardar el HTML temporalmente
html_file = "visualizacion.html"
with open(html_file, "w") as f:
    f.write(html_content)

# Abrir el archivo en una nueva pestaña del navegador
webbrowser.open("file://" + os.path.abspath(html_file))


True